In [2]:
## Test act_as_list decorator -- creating a "fake list" class
from useful_tools.act_as_list import act_as_list
@act_as_list('objects')
class MyClassThatLooksLikeAList:
    def __init__(self, objects = []):
        self.objects = objects
    def in_uppercase(self):
        return " ".join(self).upper()
    def contains_both_hello_and_world(self):
        return "hello" in self and "world" in self
fake_list = MyClassThatLooksLikeAList(["hello", "world"])
fake_list.insert(fake_list.index("world"), "the")
print(fake_list)

a = ["hello", "world"]
a.insert(a.index("world"), "the")
print(a)

fake_list = MyClassThatLooksLikeAList(["hello", "world"])
print(fake_list.in_uppercase())
print(fake_list.remove("hello"))
print(fake_list)
print(list(fake_list) == ["world"])

l = ["hello", "world"]
print(l.remove("hello"))
print(l)
print(l == ["world"])

print("--------------------")
fake_list = MyClassThatLooksLikeAList(["hello"])
x = fake_list + l # you can add a fake list and a real list, and the result will be a "fake list".
print(x)
print(fake_list.__class__) # act_as_list
print(x.__class__)         # also act_as_list
print(list(x).__class__)   # but if you want list, you can get it.

['hello', 'the', 'world']
['hello', 'the', 'world']
HELLO WORLD
None
['world']
True
None
['world']
True
--------------------
['hello', 'world']
<class 'useful_tools.act_as_list.MyClassThatLooksLikeAList'>
<class 'useful_tools.act_as_list.MyClassThatLooksLikeAList'>
<class 'list'>


In [6]:
## Test the cache_to_memory decorator
from useful_tools.cache_decorators import cache_to_memory
class MyClass:
    def __init__(self):
        self.counter = 0
    @property # you can of course skip the @property decorator and have a method that returns a value
    @cache_to_memory
    def my_property(self):
        self.counter += 1
        return f"my_property was called {self.counter} times"

my_object = MyClass()
print(my_object.my_property) # should show that my_property was called 1 time
print(my_object.my_property) # should show that my_property was called 1 time again, because it was cached

my_property was called 1 times
my_property was called 1 times


In [7]:
## Test the cache_to_disk decorator
import time
from useful_tools.cache_decorators import cache_to_disk
class MyClass:
    cache_enabled = True
    cache_dir = "cache"
    cache_expiration = 2 # seconds
    force_cache_expiration = False
    ignore_cache_expiration = False
    number_of_calls = 0
    
    @property
    @cache_to_disk
    def my_property(self):
        self.number_of_calls += 1
        return f"my_property called {self.number_of_calls} times"

myclass = MyClass()
print(myclass.my_property)  # prints "my_property called 1 times"
print(myclass.cache_status) # gives info about the use of cache in the previous call
print(myclass.my_property)  # prints "my_property called 1 times", as the result is cached
print(myclass.cache_status) # gives info about the use of cache in the previous call
time.sleep(2)
print("Waited 2 seconds...")
print(myclass.my_property)  # prints "my_property called 2 times", as the cache has expired
print(myclass.cache_status) # gives info about the use of cache in the previous call

my_property called 1 times
{'MyClass.my_property_5740354900026072187_133146708735736': ['cache_expiration_set: 2s', 'cache_file_exists', 'cache_expired: 1.2h passed', 'method_called', 'cache_saved']}
my_property called 1 times
{'MyClass.my_property_5740354900026072187_133146708735736': ['cache_expiration_set: 2s', 'cache_file_exists', 'cache_loaded']}
Waited 2 seconds...
my_property called 2 times
{'MyClass.my_property_5740354900026072187_133146708735736': ['cache_expiration_set: 2s', 'cache_file_exists', 'cache_expired: 2.008s passed', 'method_called', 'cache_saved']}


In [2]:
## test deleting the cache file following a failed/invalid response
## This is useful in order to prevent caching an invalid response
from useful_tools.cache_decorators import cache_to_disk, delete_last_saved_cache_file
class MyClass:
    cache_enabled = True
    cache_dir = "cache"
    cache_expiration = 2 # seconds
    force_cache_expiration = False
    ignore_cache_expiration = False

    @cache_to_disk
    def my_method(self):
        return "my_method"
    
    @delete_last_saved_cache_file
    def delete_cache(self):
        pass # the deletion is done in the wrapper

myclass = MyClass()
print(myclass.my_method()) # prints "my_method"
print(myclass.cache_status) # gives info about the use of cache in the previous call
# determine if the response is valid
# if the response is invalid, delete the cache file
myclass.delete_cache()
print(myclass.my_method()) # prints "my_method"
print(myclass.cache_status) # gives info about the use of cache in the previous call
print("You can see from the cache status that the cache file was deleted.")

my_method
{'MyClass.my_method_5740354900026072187_133146708735736': ['cache_expiration_set: 2s', 'cache_file_exists', 'cache_expired: 49.3s passed', 'method_called', 'cache_saved']}
my_method
{'MyClass.my_method_5740354900026072187_133146708735736': ['cache_expiration_set: 2s', 'cache_file_does_not_exist', 'method_called', 'cache_saved']}
You can see from the cache status that the cache file was deleted.
